In [1]:
# Initial imports
import os
import pandas as pd
import requests
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit
import nltk as nltk
import matplotlib.pyplot as plt
from alpaca_trade_api import StreamConn
from alpaca_trade_api.common import URL
import logging
import time
from alpaca_trade_api.stream import StreamConn
from alpaca_trade_api.common import URL

%matplotlib inline

import asyncio
import websockets

ImportError: cannot import name 'StreamConn' from 'alpaca_trade_api.stream' (C:\Users\CharbelNehme\anaconda3\envs\dev\lib\site-packages\alpaca_trade_api\stream.py)

In [ ]:
ALPACA_API_KEY  = 'PK2WA1GYRO281K7F855F'
ALPACA_SECRET_KEY = 'DjgIrZMUgqsL7E95XV4WU9fi7Lc4441WkL8ylWM6'
base_url = 'https://paper-api.alpaca.markets'
data_url = 'https://data.alpaca.markets'

In [ ]:
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)

In [ ]:
# instantiate REST API
api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url, api_version='v2')

# init WebSocket
conn = tradeapi.stream2.StreamConn(
	ALPACA_API_KEY,
	ALPACA_SECRET_KEY,
	base_url=base_url,
	data_url=data_url,
	data_stream='alpacadatav1',
)

In [ ]:
@conn.on(r'Q\..+')
async def on_quotes(conn, channel, quote):
print('quote', quote)
        
conn.run(['alpacadatav1/Q.GOOG'])

In [ ]:
from alpaca_trade_api import StreamConn
from alpaca_trade_api.common import URL

def run_connection(conn, channels):
    try:
        conn.run(channels)
    except Exception as e:
        print(f'Exception from websocket connection: {e}')
    finally:
        print("Trying to re-establish connection")
        time.sleep(3)
        run_connection(conn, channels)

if __name__ == '__main__':
    channels = ['alpacadatav1/Q.GOOG']

    conn = StreamConn(
        ALPACA_API_KEY,
        ALPACA_SECRET_KEY,
        base_url=URL('https://paper-api.alpaca.markets'),
        data_url=URL('https://data.alpaca.markets'),
        data_stream='alpacadatav1'
    )
    @conn.on(r'Q\..+')
    async def on_quotes(conn, channel, quote):
        print('quote', quote)

    run_connection(conn, channels)

In [ ]:
# Change subscription
import threading
import asyncio
import time

from alpaca_trade_api import StreamConn
from alpaca_trade_api.common import URL

conn: StreamConn = None

def consumer_thread():
    try:
        # make sure we have an event loop, if not create a new one
        loop = asyncio.get_event_loop()
        loop.set_debug(True)
    except RuntimeError:
        asyncio.set_event_loop(asyncio.new_event_loop())

    global conn
    conn = StreamConn(
        ALPACA_API_KEY,
        ALPACA_SECRET_KEY,
        base_url=URL('https://paper-api.alpaca.markets'),
        data_url=URL('https://data.alpaca.markets'),
        data_stream='alpacadatav1'
    )
    @conn.on(r'^AM\..+$')
    async def on_minute_bars(conn, channel, bar):
        print('bars', bar)


    @conn.on(r'Q\..+')
    async def on_quotes(conn, channel, quote):
        print('quote', quote)

    @conn.on(r'T\..+')
    async def on_trades(conn, channel, trade):
        print('trade', trade)

    conn.run(['alpacadatav1/Q.GOOG'])

if __name__ == '__main__':
    threading.Thread(target=consumer_thread).start()

    loop = asyncio.get_event_loop()

    time.sleep(5)  # give the initial connection time to be established
    subscriptions = [['alpacadatav1/AM.TSLA'], ['alpacadatav1/Q.GOOG'],
                     ['alpacadatav1/T.AAPL']]

    
while 1:
    for channels in subscriptions:
        loop.run_until_complete(conn.subscribe(channels))
        if "AM." in channels[0]:
            time.sleep(60)  # aggs are once every minute. give it time
        else:
            time.sleep(20)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\CharbelNehme\anaconda3\envs\dev\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\CharbelNehme\anaconda3\envs\dev\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\CharbelNehme\AppData\Local\Temp\ipykernel_12544\3474854030.py", line 21, in consumer_thread
NameError: name 'ALPACA_API_KEY' is not defined


AttributeError: 'NoneType' object has no attribute 'subscribe'

In [ ]:
@conn.on(r'^AM.AAPL$')
async def on_minute_bars(conn, channel, bar):
	if isinstance(candlesticks.df, pd.DataFrame):
		ts = pd.to_datetime(bar.timestamp, unit='ms')
		candlesticks.df.loc[ts] = [bar.open, bar.high, bar.low, bar.close, bar.volume]

	if not trade_params['trade_taken']:
		if bar.high > trade_params['high']:
			trade_params['trade_taken'] = send_order('buy', bar)

		elif bar.low < trade_params['low']:
			trade_params['trade_taken'] = send_order('sell', bar)

	if time_to_market_close() > 120:
		wait_for_market_open()

In [ ]:
@conn.on(r'^trade_updates$')
async def on_trade_updates(conn, channel, trade):
	if trade.order['order_type'] != 'market' and trade.order['filled_qty'] == '100':
		# trade closed - look for new trade
		trade_params = set_trade_params(candlesticks.df.AAPL)

In [ ]:


logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)

In [ ]:
async def print_quote(q):
    print('quote', q)

In [ ]:
"""
In this example code we wrap the ws connection to make sure we reconnect
in case of ws disconnection.
"""
import logging
import time
from alpaca_trade_api.stream import Stream
from alpaca_trade_api.common import URL

logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)

def run_connection(conn):
    try:
        conn.run()
    except KeyboardInterrupt:
        print("Interrupted execution by user")
        loop.run_until_complete(conn.stop_ws())
        exit(0)
    except Exception as e:
        print(f'Exception from websocket connection: {e}')
    finally:
        print("Trying to re-establish connection")
        time.sleep(3)
        run_connection(conn)
        
if __name__ == '__main__':
    conn = Stream(ALPACA_API_KEY,
                  ALPACA_SECRET_KEY,
                  base_url = base_url,
                  data_feed = data_url)

    conn.subscribe_quotes(print_quote, 'AAPL')

    run_connection(conn)

In [ ]:
# Pause/resume datastream
import threading
import asyncio
import time

from alpaca_trade_api import StreamConn
from alpaca_trade_api.common import URL


conn: StreamConn = None

def consumer_thread():
    try:
        # make sure we have an event loop, if not create a new one
        loop = asyncio.get_event_loop()
        loop.set_debug(True)
    except RuntimeError:
        asyncio.set_event_loop(asyncio.new_event_loop())

    global conn
    conn = StreamConn(
        ALPACA_API_KEY,
        ALPACA_SECRET_KEY,
        base_url=URL('https://paper-api.alpaca.markets'),
        data_url=URL('https://data.alpaca.markets'),
        data_stream='alpacadatav1'
    )

    @conn.on(r'^AM\..+$')
    async def on_minute_bars(conn, channel, bar):
        print('bars', bar)


    @conn.on(r'Q\..+')
    async def on_quotes(conn, channel, quote):
        print('quote', quote)


    @conn.on(r'T\..+')
    async def on_trades(conn, channel, trade):
        print('trade', trade)

    conn.run(['alpacadatav1/Q.GOOG'])

if __name__ == '__main__':
    threading.Thread(target=consumer_thread).start()

    loop = asyncio.get_event_loop()

    time.sleep(5)  # give the initial connection time to be established
    subscriptions = [['alpacadatav1/AM.TSLA'], ['alpacadatav1/Q.GOOG'],
                     ['alpacadatav1/T.AAPL']]

    while 1:
        for channels in subscriptions:
            loop.run_until_complete(conn.subscribe(channels))
            if "AM." in channels[0]:
                time.sleep(60)  # aggs are once every minute. give it time
            else:
                time.sleep(20)

In [ ]:
# Change subscription 
conn: StreamConn = None

def consumer_thread():
    try:
        # make sure we have an event loop, if not create a new one
        loop = asyncio.get_event_loop()
        loop.set_debug(True)
    except RuntimeError:
        asyncio.set_event_loop(asyncio.new_event_loop())

    global conn
    conn = StreamConn(
        ALPACA_API_KEY,
        ALPACA_SECRET_KEY,
        base_url=URL('https://paper-api.alpaca.markets'),
        data_url=URL('https://data.alpaca.markets'),
        data_stream='alpacadatav1'
    )

    @conn.on(r'^AM\..+$')
    async def on_minute_bars(conn, channel, bar):
        print('bars', bar)


    @conn.on(r'Q\..+')
    async def on_quotes(conn, channel, quote):
        print('quote', quote)


    @conn.on(r'T\..+')
    async def on_trades(conn, channel, trade):
        print('trade', trade)

    conn.run(['alpacadatav1/Q.GOOG'])

if __name__ == '__main__':
    threading.Thread(target=consumer_thread).start()

    loop = asyncio.get_event_loop()

    time.sleep(5)  # give the initial connection time to be established
    subscriptions = [['alpacadatav1/AM.TSLA'], ['alpacadatav1/Q.GOOG'],
                     ['alpacadatav1/T.AAPL']]

    while 1:
        for channels in subscriptions:
            loop.run_until_complete(conn.subscribe(channels))
            if "AM." in channels[0]:
                time.sleep(60)  # aggs are once every minute. give it time
            else:
                time.sleep(20)

In [ ]:
from enum import Enum
import time
import alpaca_trade_api as tradeapi
import asyncio
import os
import pandas as pd
import sys
from alpaca_trade_api.rest import TimeFrame, URL
from alpaca_trade_api.rest_async import gather_with_concurrency, AsyncRest

NY = 'America/New_York'


class DataType(str, Enum):
    Bars = "Bars"
    Trades = "Trades"
    Quotes = "Quotes"


def get_data_method(data_type: DataType):
    if data_type == DataType.Bars:
        return rest.get_bars_async
    elif data_type == DataType.Trades:
        return rest.get_trades_async
    elif data_type == DataType.Quotes:
        return rest.get_quotes_async
    else:
        raise Exception(f"Unsupoported data type: {data_type}")


async def get_historic_data_base(symbols, data_type: DataType, start, end,
                                 timeframe: TimeFrame = None):
    """
    base function to use with all
    :param symbols:
    :param start:
    :param end:
    :param timeframe:
    :return:
    """
    major = sys.version_info.major
    minor = sys.version_info.minor
    if major < 3 or minor < 6:
        raise Exception('asyncio is not support in your python version')
    msg = f"Getting {data_type} data for {len(symbols)} symbols"
    msg += f", timeframe: {timeframe}" if timeframe else ""
    msg += f" between dates: start={start}, end={end}"
    print(msg)
    step_size = 1000
    results = []
    for i in range(0, len(symbols), step_size):
        tasks = []
        for symbol in symbols[i:i+step_size]:
            args = [symbol, start, end, timeframe.value] if timeframe else \
                [symbol, start, end]
            tasks.append(get_data_method(data_type)(*args))

        if minor >= 8:
            results.extend(await asyncio.gather(*tasks, return_exceptions=True))
        else:
            results.extend(await gather_with_concurrency(500, *tasks))

    bad_requests = 0
    for response in results:
        if isinstance(response, Exception):
            print(f"Got an error: {response}")
        elif not len(response[1]):
            bad_requests += 1

    print(f"Total of {len(results)} {data_type}, and {bad_requests} "
          f"empty responses.")


async def get_historic_bars(symbols, start, end, timeframe: TimeFrame):
    await get_historic_data_base(symbols, DataType.Bars, start, end, timeframe)


async def get_historic_trades(symbols, start, end, timeframe: TimeFrame):
    await get_historic_data_base(symbols, DataType.Trades, start, end)


async def get_historic_quotes(symbols, start, end, timeframe: TimeFrame):
    await get_historic_data_base(symbols, DataType.Quotes, start, end)


async def main(symbols):
    start = pd.Timestamp('2021-05-01', tz=NY).date().isoformat()
    end = pd.Timestamp('2021-08-30', tz=NY).date().isoformat()
    timeframe: TimeFrame = TimeFrame.Day
    await get_historic_bars(symbols, start, end, timeframe)
    await get_historic_trades(symbols, start, end, timeframe)
    await get_historic_quotes(symbols, start, end, timeframe)


if __name__ == '__main__':
    api_key_id = os.environ.get('APCA_API_KEY_ID')
    api_secret = os.environ.get('APCA_API_SECRET_KEY')
    base_url = "https://paper-api.alpaca.markets"
    feed = "iex"  # change to "sip" if you have a paid account

    rest = AsyncRest(key_id=api_key_id,
                     secret_key=api_secret)

    api = tradeapi.REST(key_id=api_key_id,
                        secret_key=api_secret,
                        base_url=URL(base_url))

    start_time = time.time()
    symbols = [el.symbol for el in api.list_assets(status='active')]
    symbols = symbols[:200]
    asyncio.run(main(symbols))

In [ ]:
from enum import Enum
import pandas as pd
from .entity import Bar, Entity, Trade, Quote, _NanoTimestamped
from typing import Dict

trade_mapping_v2 = {
    "i": "id",
    "S": "symbol",
    "c": "conditions",
    "x": "exchange",
    "p": "price",
    "s": "size",
    "t": "timestamp",
    "z": "tape",  # stocks only
    "tks": "takerside"  # crypto only
}

quote_mapping_v2 = {
    "S":  "symbol",
    "x": "exchange",  # crypto only
    "ax": "ask_exchange",
    "ap": "ask_price",
    "as": "ask_size",
    "bx": "bid_exchange",
    "bp": "bid_price",
    "bs": "bid_size",
    "c":  "conditions",  # stocks only
    "t":  "timestamp",
    "z":  "tape"  # stocks only
}

bar_mapping_v2 = {
    "S":  "symbol",
    "x": "exchange",  # crypto only
    "o":  "open",
    "h":  "high",
    "l":  "low",
    "c":  "close",
    "v":  "volume",
    "t":  "timestamp",
    "n":  "trade_count",
    "vw": "vwap"
}

status_mapping_v2 = {
    "S":  "symbol",
    "sc": "status_code",
    "sm": "status_message",
    "rc": "reason_code",
    "rm": "reason_message",
    "t":  "timestamp",
    "z":  "tape"
}

luld_mapping_v2 = {
    "S": "symbol",
    "u": "limit_up_price",
    "d": "limit_down_price",
    "i": "indicator",
    "t": "timestamp",
    "z": "tape"
}

cancel_error_mapping_v2 = {
    "S": "symbol",
    "i": "id",
    "x": "exchange",
    "p": "price",
    "s": "size",
    "a": "cancel_error_action",
    "z": "tape",
    "t": "timestamp",
}

correction_mapping_v2 = {
    "S": "symbol",
    "x": "exchange",
    "oi": "original_id",
    "op": "original_price",
    "os": "original_size",
    "oc": "original_conditions",
    "ci": "corrected_id",
    "cp": "corrected_price",
    "cs": "corrected_size",
    "cc": "corrected_conditions",
    "z": "tape",
    "t": "timestamp",
}

orderbook_mapping_v2 = {
    "S": "symbol",
    "x": "exchange",
    "t": "timestamp",
    "b": "bids",
    "a": "asks",
}


class EntityListType(Enum):
    Trade = Trade, trade_mapping_v2
    Quote = Quote, quote_mapping_v2
    Bar = Bar, bar_mapping_v2


class EntityList(list):
    def __init__(self, entity_type: EntityListType, raw):
        entity = entity_type.value[0]
        super().__init__([entity(o) for o in raw])
        self._raw = raw
        self.mapping = entity_type.value[1]

    @property
    def df(self):
        if not hasattr(self, '_df'):
            df = pd.DataFrame(
                self._raw,
            )

            df.columns = [self.mapping.get(c, c) for c in df.columns]
            if not df.empty:
                df.set_index('timestamp', inplace=True)
                df.index = pd.DatetimeIndex(df.index)
            self._df = df
        return self._df


class Remapped:
    def __init__(self, mapping: Dict[str, str], *args, **kwargs):
        self._reversed_mapping = {
            value: key for (key, value) in mapping.items()}
        super().__init__(*args, **kwargs)

    def __getattr__(self, key):
        if key in self._reversed_mapping:
            return super().__getattr__(self._reversed_mapping[key])
        return super().__getattr__(key)


class BarsV2(EntityList):
    def __init__(self, raw):
        super().__init__(EntityListType.Bar, raw)


class TradesV2(EntityList):
    def __init__(self, raw):
        super().__init__(EntityListType.Trade, raw)


class QuotesV2(EntityList):
    def __init__(self, raw):
        super().__init__(EntityListType.Quote, raw)


class TradeV2(Remapped, _NanoTimestamped, Entity):
    _tskeys = ('t',)

    def __init__(self, raw):
        super().__init__(trade_mapping_v2, raw)


class QuoteV2(Remapped, _NanoTimestamped, Entity):
    _tskeys = ('t',)

    def __init__(self, raw):
        super().__init__(quote_mapping_v2, raw)


class BarV2(Remapped, _NanoTimestamped, Entity):
    _tskeys = ('t',)

    def __init__(self, raw):
        super().__init__(bar_mapping_v2, raw)


class StatusV2(Remapped, _NanoTimestamped, Entity):
    _tskeys = ('t',)

    def __init__(self, raw):
        super().__init__(status_mapping_v2, raw)


class LULDV2(Remapped, _NanoTimestamped, Entity):
    _tskeys = ('t',)

    def __init__(self, raw):
        super().__init__(luld_mapping_v2, raw)


class CancelErrorV2(Remapped, _NanoTimestamped, Entity):
    _tskeys = ('t',)

    def __init__(self, raw):
        super().__init__(cancel_error_mapping_v2, raw)


class CorrectionV2(Remapped, _NanoTimestamped, Entity):
    _tskeys = ('t',)

    def __init__(self, raw):
        super().__init__(correction_mapping_v2, raw)


class SnapshotV2:
    def __init__(self, raw):
        self.latest_trade = _convert_or_none(TradeV2, raw.get('latestTrade'))
        self.latest_quote = _convert_or_none(QuoteV2, raw.get('latestQuote'))
        self.minute_bar = _convert_or_none(BarV2, raw.get('minuteBar'))
        self.daily_bar = _convert_or_none(BarV2, raw.get('dailyBar'))
        self.prev_daily_bar = _convert_or_none(BarV2, raw.get('prevDailyBar'))


class SnapshotsV2(dict):
    def __init__(self, raw):
        for k, v in raw.items():
            self[k] = _convert_or_none(SnapshotV2, v)


class LatestBarsV2(dict):
    def __init__(self, raw):
        for k, v in raw.items():
            self[k] = _convert_or_none(BarV2, v)


class LatestTradesV2(dict):
    def __init__(self, raw):
        for k, v in raw.items():
            self[k] = _convert_or_none(TradeV2, v)


class LatestQuotesV2(dict):
    def __init__(self, raw):
        for k, v in raw.items():
            self[k] = _convert_or_none(QuoteV2, v)


class BidOrAsk(Entity):
    def __init__(self, raw):
        super().__init__(raw)


class OrderbookV2(Entity):
    def __init__(self, raw):
        super().__init__(raw)
        if self.bids:
            for i in range(len(self.bids)):
                self.bids[i] = BidOrAsk(self.bids[i])
        if self.asks:
            for i in range(len(self.asks)):
                self.asks[i] = BidOrAsk(self.asks[i])


class NewsV2(Entity):
    def __init__(self, raw):
        super().__init__(raw)


class NewsListV2(list):
    def __init__(self, raw):
        super().__init__([NewsV2(o) for o in raw])


def _convert_or_none(entityType, value):
    if value:
        return entityType(value)
    return None

# LIVE DATA STREAM

In [ ]:
# Alpaca for data
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame

import plotly.graph_objects as go
import plotly.express as px


# Setup instance of alpaca api
alpaca = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)

# instance of data streaming API
alpaca_stream = Stream(ALPACA_API_KEY, ALPACA_SECRET_KEY)

# handler for receiving bar data for Bitcoin
async def on_crypto_bar(bar):
    if bar.exchange != 'CBSE':
        return

print(bar)
   
# handler for receiving bar data for Coinbase stock
async def on_equity_bar(bar):
    print(bar)

# subscribe to coinbase stock data and assign handler
alpaca_stream.subscribe_bars(on_equity_bar, "COIN")


# subscribe to Bitcoin data and assign handler
alpaca_stream.subscribe_crypto_bars(on_crypto_bar, "BTCUSD")

# start streaming of data
alpaca_stream.run()

In [ ]:
# change handlers
 
async def on_equity_bar(bar):
   # bar data passed to intermediary function
   synch_datafeed(bar)
 
async def on_crypto_bar(bar):
   if bar.exchange != 'CBSE':
       return
   # bar data passed to intermediary function
   synch_datafeed(bar)

In [ ]:
# change handlers
 
async def on_equity_bar(bar):
   # bar data passed to intermediary function
   synch_datafeed(bar)
 
async def on_crypto_bar(bar):
   if bar.exchange != 'CBSE':
       return
   # bar data passed to intermediary function
   synch_datafeed(bar)

In [ ]:
data = {}
 
def synch_datafeed(bar):
   # convert bar timestamp to human readable form
   time = datetime.fromtimestamp(bar.timestamp / 1000000000)
   symbol = bar.symbol
 
   # If we’ve never seen this timestamp before
   if time not in data:
       # Store bar data in a dictionary keyed by symbol and wait for data from the other symbol
       data[time] = {symbol:bar}
       return
    
    # If we’ve reached this point, then we have data for both symbols for the current timestamp
    data[time][symbol] = bar
   
    # retrieve dictionary containing bar data from a single timestamp
    timeslice = data[time]
  	
   # pass that data into our next function for processing
   on_synch_data(timeslice)

In [ ]:
def on_synch_data(data):
   # access bar data
   btc_data = data["BTCUSD"]
   coin_data = data["COIN"]
   
   # save reference of close data for each bar
   btc_close = btc_data.close
   coin_close = coin_data.close
	
   # calculate spread 
   spread = btc_close - coin_close
 
   # we will use the historical STD
   spread_std = historical_spread_std
   # calculate entry and exit levels for standard deviation
   entry_level = -1 * spread_std
   loss_exit_level = -3 * spread_std
   
   # pass spread and level data to next part for placing trades
   place_trades(spread, enty_level, loss_exit_level)

In [ ]:
def place_trades(spread, entry_level, loss_exit_level):
 
	# there is an active position if there is at least 1 position
      active_position = len(alpaca_trade.list_positions()) != 0    
 
if spread < entry_level and not active_position:
     # retrieve buying power from account details
     buying_power = alpaca_trade.get_account().buying_power
           # the buying power allocated to each asset will be half of the total
	     btc_notional_size = buying_power // 2
           coin_notional_size = buying_power // 2
           
          # place long order on BTCUSD
           alpaca_trade.submit_order(symbol="BTCUSD", notional=btc_notional_size, type='market', side='buy', time_in_force='day')
 
          # Place short order for COIN
           alpaca_trade.submit_order(symbol="COIN", notional=coin_notional_size, type='market', side='sell', time_in_force='day')
 
   elif spread < loss_exit_level and active_position:
       # liquidate if loss exit level is breached
       alpaca_trade.close_all_positions()
 
   elif spread > 0 and active_position:
       # liquidate if 0 spread is crossed with an active position
       alpaca_trade.close_all_positions()